In [146]:
from copy import copy
import re
from string import punctuation
import charset_normalizer
import os
from tqdm import tqdm
import json
import networkx as nx
from networkx.readwrite import json_graph
import pymorphy2
import pandas as pd

# ENCODING

The texts have been extracted with different tools and saved in different encodints

In [229]:
TXT_DIRECTORY = 'D:\Диплом_текстовые_квесты\Data\TXT\part1'

In [10]:
for file in tqdm(os.listdir(TXT_DIRECTORY)):
    file_path = os.path.join(TXT_DIRECTORY, file)
    with open(file_path, 'rb') as f:
        file_content = f.read()
        detected_encoding = charset_normalizer.detect(file_content)['encoding']
    if detected_encoding != 'utf_8':
        if not detected_encoding:
            detected_encoding='ansi'
        with open(file_path, encoding=detected_encoding) as f:
            text = f.read()
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(text)

100%|██████████████████████████████████████████████████████████████████████████████████| 37/37 [00:34<00:00,  1.06it/s]


In [149]:
Problem files were resaved manually

'BB1rusKalgor.txt'

# DISCHARGE

    Some quests contain discharge('смотрите, это т е к с т  с  р а з р я д к о й'), which leads to incorrect tokenization and should be removed

In [11]:
def remove_discharge(lines:list, file2log=None):
    clean_lines = []
    for line in lines:
        discharged_fragments = []
        current_discharged_seq = []
        #two spaces are used as a words separator in discharged fragments
        if '  ' not in line:
            clean_line = line
        else:
            clean_line = copy(line)
            pseudotokens = line.split()
            #collect discharged sequences without joining it
            for token in pseudotokens:
                if len(token.strip(punctuation + '«»')) == 1 and token.strip(punctuation + '«»').isalpha():
                    current_discharged_seq.append(token)
                else:
                    if len(current_discharged_seq) > 3:
                        discharged_fragments.append(current_discharged_seq)
                    current_discharged_seq = []
            if len(current_discharged_seq) > 3:
                discharged_fragments.append(current_discharged_seq)
            if discharged_fragments:
                for discharged_fragment_tokens in discharged_fragments:
                    try:
                        fragment_regexp = '\s+'.join(discharged_fragment_tokens)
                    #get the fragment with the same token separators as in the source text
                        fragment_match = re.search(fragment_regexp, clean_line)
                        if fragment_match:
                            fragment = fragment_match.group()
                        #replace the fragment with the same text without extra separators
                            discharged_tokens = re.split('\s{2,}', fragment)
                            clean_tokens = [''.join(token.split()) for token in discharged_tokens if token]
                            clean_fragment = ' '.join(clean_tokens)
                            clean_line = re.sub(fragment, clean_fragment, clean_line)
                    except:
                        print(file2log)
                        print(line)
        clean_lines.append(clean_line) 
        
    return clean_lines
                

In [12]:
test_lines = ['ты знаешь с  н а с т у п л е н ь е м   т е м н о т ы',
             'п ы т а ю с ь  я отсчитывать н а  г л а з,',
             'о т с ч и т ы в а я  горе от версты,',
             'пространство,  р а з д е л я ю щ е е  нас',
             'И числа как-то сходятся в слова,',
              'Откуда  приближаются ко мне']

In [134]:
remove_discharge(test_lines)

['ты знаешь с наступленьем темноты',
 'пытаюсь я отсчитывать на глаз,',
 'отсчитывая  горе от версты,',
 'пространство,  разделяющее  нас',
 'И числа как-то сходятся в слова,',
 'Откуда  приближаются ко мне']

In [13]:
for file in tqdm(os.listdir(TXT_DIRECTORY)):
    file_path = os.path.join(TXT_DIRECTORY, file)
    with open(file_path, encoding='utf-8') as f:
        lines = f.readlines()
    clean_lines = remove_discharge(lines, file)
    with open(file_path, 'w', encoding='utf-8') as f:
        f.write(''.join(clean_lines))

100%|██████████████████████████████████████████████████████████████████████████████████| 37/37 [00:00<00:00, 45.35it/s]


# Check punctuation problems

In some files punctuation is misplaced at the end of the string. 

In [156]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [170]:
def punctuation_tends_to_move_to_the_line_end(file_path, num_punctuation_threshold=2, bad_text_threshold=0.3, encoding='utf-8'):
    with open(file_path, encoding=encoding) as f:
        lines = f.readlines()
    num_bad_lines = 0
    num_meaningfull_lines = len([line for line in lines if len(line) > num_punctuation_threshold])
    for line in lines:
        symbols = re.findall('[\S]', line)
        if len(symbols)> num_punctuation_threshold and all([symbol in punctuation+'«»' for symbol in symbols[-num_punctuation_threshold:]]):
            num_bad_lines += 1
    return num_bad_lines >= bad_text_threshold*num_meaningfull_lines and num_meaningfull_lines

In [172]:
for file in os.listdir(TXT_DIRECTORY):
    if punctuation_tends_to_move_to_the_line_end(os.path.join(TXT_DIRECTORY, file)):
        print(file)

Выбор сотника.txt
За семью печатями _v1.1.txt
Заложники пиратского адмирала.txt
Замок потерянных душ - ч3.txt
Кристал Каспли.txt
Лес и трава.txt
Ловушка - часть 1 (v1.01, A5).txt
Наперегонки с торнадо.txt


These files will be reextracted using another tool

# Unification of the numeration position

Most of the files contains text fragments with the ids placed at lines like headings. There are several heading parts which are not parts of ids, let's unificate it to make it possible to simplify splitting the text into fragments

In [14]:
def strip_id_lines_in_file(file_path, fragments_to_remove=['-', '\.', '№', '\*', '§', 'Глава', 'глава', 'п', 'ПАРАГРАФ'], encoding='utf-8'):
    with open(file_path, encoding=encoding) as f:
        lines = f.readlines()
    new_lines = []
    for line in lines:
        try:
            line_without_fragments_to_remove = copy(line)
            for fragment in fragments_to_remove:
                line_without_fragments_to_remove = re.sub(fragment, '', line_without_fragments_to_remove)
            new_lines.append(str(int(line_without_fragments_to_remove))+'\n')
        except ValueError:
            new_lines.append(line)
    with open(file_path, 'w', encoding=encoding) as f:
        f.write(''.join(new_lines))

In [15]:
for file in tqdm(os.listdir(TXT_DIRECTORY)):
    strip_id_lines_in_file(os.path.join(TXT_DIRECTORY, file))

100%|██████████████████████████████████████████████████████████████████████████████████| 37/37 [00:02<00:00, 12.61it/s]


Let's detect files, there the id-like line is not the first non-empty one, and assert, that 

In [16]:
def starts_with_id(file_lines:list):
    for line in file_lines:
        if line.strip():
            try:
                number = int(line)
                return True
            except ValueError:
                return False

In [17]:
def ids_are_ordered(file_lines:list):
    are_ordered = True
    previous_id = -1
    for line in file_lines:
        try:
            number = int(line)
            if number > previous_id:
                previous_id = number
            else:
                are_ordered = False
                print(number, previous_id)
                previous_id = number
        except ValueError:
            continue
    return are_ordered

In [18]:
not_ordered_files = []
for file in os.listdir(TXT_DIRECTORY):
    file_path = os.path.join(TXT_DIRECTORY, file)
    with open(os.path.join(TXT_DIRECTORY, file_path), encoding='utf-8') as f:
        lines = f.readlines()
    if not ids_are_ordered(lines):
        print(file)
        not_ordered_files.append(file)

27 62
42 68
43 78
Растревоженный Манхеттен.txt
37 87
142 158
Резонанс (светлая версия).txt
6 55
38 3737
45 4444
50 4949
62 6161
68 6767
73 7272
86 8585
88 8787
92 9191
94 497
99 9898
110 108108
115 113113
117 116116
145 143143
151 150150
165 163163
175 173173
187 185185
194 192192
200 199199
206 205205
242 240240
246 243243
257 256256
272 271271
278 276276
282 280280
302 299299
306 305305
314 313313
320 318318
324 323323
341 339339
355 354354
359 357357
372 370370
397 395395
400 399399
412 411411
416 414414
421 419419
430 428428
444 443443
451 449449
465 463463
468 467467
471 469469
479 478478
489 487487
499 497497
523 522522
527 525525
537 534534
541 538538
543 542542
548 547547
560 559559
573 572572
575 574574
583 580580
591 590590
614 611611
625 624624
640 637637
642 641641
645 643643
Робур2 - Город Вин 2.txt
39 46
44 94
49 65
58 66
70 72
40 99
Рубка_драка.txt
1 2017
Рыцари Погибели (1.0).txt
6 224
232 301
1 566
1 1
1 3
1 2
1 5
1 1
2 2
1 3
1 2
1 2
1 1
1 3
1 2
1 5
1 1
2 2
1 3
1 2
Рыц

In [19]:
for file in ['Наперегонки с торнадо.txt']:
    file_path = os.path.join(TXT_DIRECTORY, file)
    with open(os.path.join(TXT_DIRECTORY, file_path), encoding='utf-8') as f:
        lines = f.readlines()
    if not ids_are_ordered(lines):
        print(file)

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Диплом_текстовые_квесты\\Data\\TXT\\part2\\Наперегонки с торнадо.txt'

In [57]:
not_ordered_files

['BB1rusKalgor.txt',
 'Cherniy continent.txt',
 'dq1_tlos_v1.0.txt',
 'dzhungar_v1.0.txt',
 'Fabled_Lands_5_The_Court_Of_Hidden_Faces_RUS.txt',
 'Fabled_Lands_6_Lords_of_the_Rising_Sun_RUS.txt',
 'Face2face.txt',
 'Fallout II v1.04.5.txt',
 'FF05 - Город воров (v1.01).txt',
 'FF21 - Глаз дракона.txt',
 'ff44_rus.txt',
 'FF7 (А5, версия 1.0).txt',
 'fftd_x1.txt',
 'fu (v1.1).txt',
 'GD-demo.txt',
 'logovo.txt',
 'serdcelda.txt',
 'SW4 (A5, v1.0).txt',
 'SW5 (A5, v1.0).txt',
 'Vasiliy_Pupkin.txt',
 'Адское болото.txt',
 'Алдар Косе приключения безбородого обманщика v4.2.txt',
 'Башня Ужаса (A4, 1.0).txt',
 'Боевые ямы Крарта.txt',
 'Болотная лихорадка (v1.0).txt',
 'Булава Ужаса.txt',
 'В поисках рождества.txt',
 'В стране высоких трав.txt',
 'Владыка степей (1.2.1b).txt',
 'Возвращение.txt',
 'Воин острого клинка.txt',
 'Вой оборотня.txt',
 'Воля зоны.txt',
 'Выбор сотника.txt',
 'Генезис 1.6.txt',
 'Города Золота и Славы.txt',
 'Демоны бездны.txt',
 'Дом Дьявола.txt',
 'За семью печатя

In [55]:
with open(os.path.join(TXT_DIRECTORY, 'dq1_tlos_v1.0.txt'), encoding='utf-8') as f:
    lines = f.readlines()
needs_checking = not ids_are_ordered(lines)

93 154
2 93
153 227
4 153
20 27
5 34
6 129
7 72
8 60
9 238
10 255
11 23
16 75
6 16
13 34
78 200
14 78
5 14
15 129
16 163
17 128
18 149
19 26
53 245
42 53
20 42
21 34
14 28
22 129
23 148
17 23
24 30
11 24
25 259
32 39
3 32
20 27
28 34
14 129
30 44
17 30
32 163
3 39
34 94
35 51
153 153
36 153
38 77
39 145
32 39
3 32
41 83
42 170
11 245
44 253
46 62
6 175
48 171
49 246
50 80
36 50
52 144
53 170
42 245
11 42
24 67
226 237
205 226
55 259
56 96
46 56
6 46
58 164
60 127
61 91
62 235
40 62
64 241
65 82
50 65
61 66
67 79
54 67
11 54
46 85
57 164
55 70
72 181
86 238
8 86
101 137
74 101
75 154
11 75
21 95
58 261
61 79
82 234
84 144
42 245
11 42
114 132
46 114
8 238
88 195
1 173
91 166
48 270
93 155
94 167
81 94
21 95
97 119
98 156
99 222
250 256
100 250
101 127
102 142
103 126
104 273
153 153
106 153
107 296
99 107
101 137
110 123
68 110
46 68
112 249
1 112
73 223
108 177
68 114
46 68
153 301
116 153
47 254
118 207
119 143
120 151
121 266
122 133
123 140
124 146
125 228
96 125
107 126
127 136
128

In [42]:
needs_checking

True

In [20]:
need_beginning_cleaning = []
need_order_checking = []
for file in os.listdir(TXT_DIRECTORY):
    file_path = os.path.join(TXT_DIRECTORY, file)
    with open(file_path, encoding='utf-8') as f:
        lines = f.readlines()
    if not starts_with_id(lines):
        need_beginning_cleaning.append(file)
    if not ids_are_ordered:
        need_order_checking.append(file)

In [21]:
need_beginning_cleaning

['Рыцари Погибели (1.0).txt']

In [40]:
need_order_checking

[]

As all the ids are ordered, let's ignore all the lines before the first id

In [233]:
def get_id4number(line:str, previous_id=None, check_order=True):
    try:
        number = int(line)
        if type(previous_id) not in [str, int] or number == int(previous_id)+1 or not check_order:
            return line.strip()
        else:
            return None
    except Exception as e:
        return None

In [88]:
res = get_id4number('1')

In [79]:
if type(None):
    print('exists')

exists


In [231]:
def split_quest(file_path, detecting_id_function, needs_order_checking=False, encoding='utf-8'):
    with open(file_path, encoding=encoding) as f:
        lines = f.readlines()
    id2fragment = {}
    current_fragment_lines = []
    current_id = None
    previous_id = None
    for line in lines:
        if needs_order_checking:
            id_ = detecting_id_function(line, previous_id)
        else:
            id_ = detecting_id_function(line)
        if id_:
            if current_id:
                id2fragment[current_id] = ''.join(current_fragment_lines)
            current_fragment_lines = []
            current_id = id_
        else:
            current_fragment_lines.append(line)
    if current_id:
        id2fragment[current_id] = ''.join(current_fragment_lines)
    return id2fragment

In [216]:
FRAGMENTS_JSON_DIRECTORY = 'D:\Диплом_текстовые_квесты\Data\questbook_fragments_json'

In [94]:
#These quests have different format of id
exceptions = ['Колдунья и книга заклинаний.txt', 'буйвол.txt']
order_exceptions = ['Наперегонки с торнадо.txt']

In [74]:
not_splitted_quests = []

In [27]:
exceptions = []

In [234]:
for file in os.listdir(TXT_DIRECTORY):
    json_name = os.path.splitext(file)[0] + '.json'
    if json_name in problem_files:
        needs_order_checking = False
        fragments = split_quest(os.path.join(TXT_DIRECTORY, file), get_id4number, needs_order_checking)
        if fragments:
            json_path = os.path.join(FRAGMENTS_JSON_DIRECTORY, json_name)
            with open(json_path, 'w', encoding='utf-8') as f:
                json.dump(fragments, f)
        else:
            not_splitted_quests.append(file)

In [103]:
for file in ['Прорыв (v1.0).txt']:
    if file not in exceptions:
        fragments = split_quest(os.path.join(TXT_DIRECTORY, file), get_id4number, needs_order_checking=True)
        if fragments:
            json_name = os.path.splitext(file)[0] + '.json'
            json_path = os.path.join(FRAGMENTS_JSON_DIRECTORY, json_name)
            with open(json_path, 'w', encoding='utf-8') as f:
                json.dump(fragments, f)
        else:
            not_splitted_quests.append(file)

In [104]:
not_splitted_quests

[]

Two quests are enumerated at the beginning of the same line. Let's  change the enumeration format and redo splitting

In [321]:
def split_enumeration_and_text(file_path, encoding='utf-8'):
    with open(file_path, encoding=encoding) as f:
        lines = f.readlines()
    new_lines = []
    for line in lines:
        id_match = re.match('\s*\d+', line)
        if not id_match:
            new_lines.append(line)
        else:
            new_lines.append(line[:id_match.span()[1]].strip()+'\n')
            new_lines.append(line[id_match.span()[1]:].lstrip(' .'))
    with open(file_path, 'w', encoding=encoding) as f:
        f.write(''.join(new_lines))

In [322]:
for file in set(not_splitted_quests):
    file_path = os.path.join(TXT_DIRECTORY, file)
    split_enumeration_and_text(file_path)

In [324]:
for file in set(not_splitted_quests):
    fragments = split_quest(os.path.join(TXT_DIRECTORY, file), get_id4number)
    if fragments:
        json_name = os.path.splitext(file)[0] + '.json'
        json_path = os.path.join(FRAGMENTS_JSON_DIRECTORY, json_name)
        with open(json_path, 'w', encoding='utf-8') as f:
            json.dump(fragments, f)
    else:
        print('file' + ' is not splitted')

In [325]:
exceptions

['Колдунья и книга заклинаний.txt', 'буйвол.txt']

In [380]:
def get_id_other_format(line, id_regexp, num_tokens_before=[0], symbols2strip=None):
    stripped_line = line.strip(symbols2strip)
    tokens = stripped_line.split()
    if len(tokens)-1 in num_tokens_before:
        stripped_line = tokens[-1]
        id_match = re.match(id_regexp, stripped_line)
        if id_match and id_match.span()[1] == len(stripped_line):
            return stripped_line
        else:
            return None
    else:
        return None

In [374]:
exceptions

['Колдунья и книга заклинаний.txt', 'буйвол.txt']

In [378]:
file2id_detecting_fuction = {
   'Колдунья и книга заклинаний.txt': lambda line: get_id_other_format(line, id_regexp='\d[А-Я]|\d+'),
   'буйвол.txt': lambda line: get_id_other_format(line, id_regexp='\d+[A-Z]', num_tokens_before=[1])
}

In [381]:
for file in exceptions:
    fragments = split_quest(os.path.join(TXT_DIRECTORY, file), file2id_detecting_fuction[file])
    if fragments:
        json_name = os.path.splitext(file)[0] + '.json'
        json_path = os.path.join(FRAGMENTS_JSON_DIRECTORY, json_name)
        with open(json_path, 'w', encoding='utf-8') as f:
            json.dump(fragments, f)
    else:
        print(file)

# Detect transitions

In [235]:
file2id_regexp = {
    'Колдунья и книга заклинаний':'\d[А-Я]|\d+',
    'буйвол': '\d+[A-Z]',
    
}

In [236]:
file2previous_tokens = {
    'Аргинк (A5, v1.0)': ['параграф'],
    'BB1rusKalgor': ['прочти'],
    'Black_Panther_v1.2': [r'(', r')'],
    'dogs2':[r'('],
    'Идущий на смех': ['–'],
    'fairysh': [r'('],
    'Face2face': ['(', ')', '-'],
    'FF#2 - Цитадель хаоса (a5, v1.1)': [r'('],
    'fu (v1.1)': [r'('],
     'Inspektor': [r'('],
    'Korablekrushenie': [r'('],
    'Labirint_zhdyot_vas': ['п'],
    'lw0_1.00_a4':[r'['],
    'SW4 (A5, v1.0)':['раздел'],
    'Адское болото': ['на'],
    'Алькатрас - П.Горохов': [r'('],
    'Боевые ямы Крарта': ['параграф'],
    'Большое приключение малыша Лешеньки': ['глава'],
    'Аргинк (A5, v1.0)': ['параграфу'],
     'Боевые ямы Крарта': ['параграф'],
    'буйвол': [r'(', r')'],
    'В землях орков': [r'(', 'далее', 'параграф'],
    'В поисках рождества': ['Прочти'],
    
    'Вереница миров': ['п'], 
    'Воин острого клинка':[r'(', '-'],
    'Выбор сотника': [r'('],
    'Выжить и вернуться домой': [r'('],
    'Генезис 1.6': [r'('],
    'Герой по случайности. Приключение первое': [r'('],
    'Город бешеных псов': [r'('],
    'Города Золота и Славы': ['параграф'],
    'Даманский': [r'('],
    'Демоны бездны':['на'],
    'Дом Дьявола':[r'('],
    'Емельянова__Искушение_студента': [r'('],
     'Жаброносец':[r'('],
    'За семью печатями _v1.1' :['§'],
    'Замок потерянных душ - ч1':['параграф'],
    'Замок потерянных душ - ч2':['параграф'],
    'Замок потерянных душ - ч3':['параграф'],
    'Замок потерянных душ - ч4':['параграф'],
    'Каникулы в джунглях':['страницу'],
    'Капитан Морской ведьмы': ['п'],
    'Катарсис - v1.5': [r'('],
    'Кинжалы во тьме': ['-'],
    'книга-игра Та самая игра': ['-'],
     'Колдунья и книга заклинаний': ['на'],
    'Кольцо джинна':  ['параграф'],
    'Кристал Каспли':[r'('],
    'Кровавые кости':[r'(', '-'],
     'Курортный роман': [r'(', '-', 'use_lines_of_numbers'],
    'Лабиринт страха': ['на'],
    'Лекарство от смерти 1.01': ['<', '>'],
    'Лес и трава': ['('],
    'Ловушка - часть 1 (v1.01, A5)': [r'('],
    'Ловушка - часть 2 (A5)': [r'('],
    'Лунатик': [r'(', 'то'],
    'Майкл Фрост - Чернолесье': ['на'],
    'Месть Альтея': [r'(', 'на'],
    'Меч самурая': ['на'],
    'Микрорыцарь': ['параграф'],
    'Морская лихорадка (v1.0, A5)': [r'(', r')'],
    'Морские байки (1.2)': ['параграф'],
    'Наемная убийца (v1.0)': ['на'],
    'Непредвиденный пассажир': ['<', '>'],
    'Ночь в лесу оборотней': ['страницу'],
    'Один против пламени': ['пункту', 'пункт', 'к'],
    'Одинокий волк 2 - Огонь на воде': ['на', "то"],
    'Операция Кладенец':['главу'],
    'Оружие возмездия - книга-игра': ['№'],
    'Остров крови': ['-'],
    'Остров неупокоенных': ['('],
    "Остров циклопа": ["на", "параграф"],
    "Остров чёрного колдуна":  [r'('],
    "Пиратская одиссея": ["на"],
    "Пленник чудо-дерева": [r'(', '-', '—'],
    "По воле Рима": ['-'],
    "Побег": [r'('],
    "Погоня":['-'],
    "Подземелья знака кошмаров":[r'(', '-'],
    "Подземная дорога 1.1":[r"["],
    "Притча о том, как Пич Квачо генерала спас":[r'(', '-'],
    
    "Пророчество о черной династии":['-'],
    "Наперегонки с торнадо": ['страница'],
    "Равнины завывающей тьмы (A5, v1.00)": ['на'],
    "Растревоженный Манхеттен":[r'(', '-'],
    "Резонанс (светлая версия)":['-'],
    "Роберт Стайн - Дневник сумасшедшей мумии": ["страницу", 'страница'],
    "Робинзон Крузо книга-игра": ['-', 'на', 'то'],
    "Робур2 - Город Вин 2": [r'('],
    'Рубка_драка': ['параграф'],
    'Рыцари Погибели (1.0)': [r'(', '-'],
    'Рыцарь живых мертвецов (v1.0)': ['-', 'на', 'то', r'('],
    'Саксонские войны':['параграф'],
    'Семь змей': ['(', '-'],
    'Сказание_о_Загоре': [r'(', '-'],
    'Скала ужаса': [r'('],
    'Скипетр Нижнего мира': ['параграф'],
    'Склеп вампира': ['параграф'],
    'Смертоносная тень - Часть 1 _': ['на'],
    'Спасаясь от тьмы': ['на', 'к'],
    'Спектральные сталкеры':  [r'(', '-'],
    'Тачанка - книга-игра': ['-'],
    'Темный узурпатор1': ['параграф'],
    'Точка отсчета': ['параграф', 'NEXT_WORD'],
    'Три дороги (v1.1)': [r'('],
    'Турнир юнлингов (v1.0, A5)':  [r'('],
    'Ты-удобрение для растений': ['страница'],
    'Угроза с Дикого Кладбища':['то', 'на', r'('],
    'Усыпальницы Королей':['прочтите', 'читайте', 'на'],
    'Филипп Эбли - Остров осьминогов - А5': ['пункт'],
    'Холмы Шамутанти':[r'(', '-', 'LINE'],
    'Храм ужаса': [r'(', '-', r'?'],
    'Цирк - западня': ['страница'],
    'Человекосжималки': ['страница'],
    'Шерлок Холмс 3 (без фона) v1.1':['на', r'('],
    'Школа для смельчаков':['на', '-'],
    'Эдвард Паккард - Тайна заброшенного замка': ['-'],
    'Элгар Флетч': ['прочти']
}
    

In [237]:
file2prohibited_neighbours = {
   'Fabled_Lands_6_Lords_of_the_Rising_Sun_RUS' : ['шаблов'],
    'Алдар Косе приключения безбородого обманщика v4.2': ['уровень'],
    'FF21 - Глаз дракона': ['выносливость', 'мастерство', 'удача'],
     'dzhungar_v1.0': ['+'],
    'Владыка степей (1.2.1b)': ['глоток'],
    'Вой оборотня': ['мастерство', 'выносливость', 'золотой'],
     'Воля зоны': ['репутация', "воля", "здоровье"],
    'КвестРЕЛИКВИЯ': ['='],
    'Кощеева цепь': ['сила'],
    'Наступление тьмы': ['мастерство', 'выносливость'],
    "Портал зла":['мастерство', 'выносливость'],
    "Проект1 - 3.0.1":["IGNORE_CAPS"],
    "Проклятый лес (A5, v1.0)": ['мастерство', 'выносливость', 'очко'],
    "Проклятие мумии": ['навык', 'стойкость'],
    "Робур2 - Город Вин 2": ['д'],
    'Тачанка - книга-игра': ['червонец', 'лекарство', 'выносливость', 'еда'],
    'Цыркунов__Ассасин': ['ед'],
    }

In [ ]:
quests_with_lines_for_transitions = {
    'dafin1',
    'dq1_tlos_v1.0',
    'Face2face',
    'fftd_x1',
    'logovo',
    'Возвращение',
    'Ночь по ту сторону Смерти (A5, v1.0)''
    'Общество червей (v1.0)',
    'Один против Империи',
    'По Кровавому морю (A5, v1.00beta)',
}

In [7]:
#default item is a sentence
#pod
file2transition_item = {
    'BB1rusKalgor': 'line',
    'Black_Panther_v1.2': 'line',
    'dafin1': 'line',
    'dogs2':'line'
    
}

In [9]:
morph = pymorphy2.MorphAnalyzer()

In [238]:
morph = pymorphy2.MorphAnalyzer()


ecranising = {'(': '\(',
              ')': '\)',
              '[': '\[',
              '?': '\?'
             }   

unecranising = {value:key for key, value in ecranising.items()}


def check_context(position, tokens, quest_name):
    possible_previous = file2previous_tokens.get(quest_name)
    prohibited_neighbours = file2prohibited_neighbours.get(quest_name)
    
    previous_token = unecranising[tokens[position-1]] if tokens[position-1] in unecranising else tokens[position-1]
    
    pretext_is_correct = (not possible_previous) or (position>0 and (previous_token.lower() in possible_previous or morph.parse(previous_token)[0].normal_form in possible_previous))
    if pretext_is_correct:
        no_prohibited_neighbours = not prohibited_neighbours or position == len(tokens)-1 or (tokens[position+1] not in prohibited_neighbours and morph.parse(tokens[position+1])[0].normal_form not in prohibited_neighbours)
        if no_prohibited_neighbours:
            return True
        
        
def book_fragments2graph(book_fragments:dict, quest_name:str):
    G = nx.DiGraph()
    
    if quest_name in file2id_regexp:
        transition_regexp = file2id_regexp[quest_name]
        get_id = lambda x: x
    else:
        transition_regexp = '\d+'
        get_id = lambda x: int(x)
        
    for node_id, text in book_fragments.items():
        G.add_nodes_from([get_id(node_id)], fragment_text = text)
    
    
    
    possible_previous = file2previous_tokens.get(quest_name)
                                   
    for node_id, text in book_fragments.items():
        node_from = get_id(node_id)
        text = re.sub('–', '-', text)
        text = re.sub('\.', '', text)
        text = re.sub(',', ' ,', text)
        text = re.sub('\)', ' \)', text)
        text = re.sub('\]', ' \]', text)
        if possible_previous:
            for previous_token in possible_previous:
                if previous_token != '-':
                    token = ecranising[previous_token] if previous_token in ecranising else previous_token
                    try:
                        text = re.sub(token, ' '+token+' ', text)
                    except:
                        print(previous_token)
        #print(text)
        tokens = text.split()
        for i, token in enumerate(tokens):
            match = re.match(transition_regexp, token)
            if match and len(token) == match.span()[1] and check_context(i, tokens, quest_name):
                node_to = get_id(token)
                G.add_edge(node_from, node_to)
                                   
        if possible_previous and 'LINE' in possible_previous:
            lines = text.split('/n')
            for line in lines:
                tokens = [token.strip() for token in line.split() if token]
                tokens = [token[1:] if token[0] == r'\\' and len(token) > 1 else token for token in tokens]
                if len(tokens) > 2 and all([re.match(transition_regexp, token) and re.match(transition_regexp, token).span()[1] for token in tokens]):
                    for token in tokens:
                        node_to = get_id(token)
                        G.add_edges(node_from, node_to)
    return G

In [205]:
file = os.path.join(FRAGMENTS_JSON_DIRECTORY, 'Black_Panther_v1.2.json')

In [192]:
fragments

{'1': 'Уже совсем стемнело, и только тонкая оранжевая полоска на горизонте напоминала о том, куда ушло\nсолнце. Небольшой кораблик уверенно шел по тихому морю. А вот в одной из кают было совсем не тихо...\n— Слушай, этой девчонки нет в базе! — проговорил парень, устало откинувшись на спинку стула. — Что\nделать будем?\n— Как нет? — вскинулся второй. — Она что, из невыездных? Так какого черта мы ее на борт взяли? Ты\nчем раньше смотрел?\n— Я что, дебил, по-твоему? — обиделся первый. — Пробил я её по невыездным, со стакана отпечатки\nснял, всё путем... По всей базе гонять не стал, времени не было. А сейчас найти не могу, ее ни в одном разделе\nнет.\n— Ага, а плохому танцору яйца мешают, — хмыкнул второй и сел за компьютер. — Учись, салага!\nСпустя час первый ехидно уточнил:\n— Так что ты там говорил насчет танцоров?\nВторой неохотно признал:\n— Нету. Значит, сбой... Ладно, пиши отчет о технической невозможности идентификации, приложи\nфото и отпечатки, а заодно пошли запрос, пусть технар

In [199]:
file = 'dogs2.json'
quest_name = os.path.splitext(file)[0]
fragments_path = os.path.join(FRAGMENTS_JSON_DIRECTORY, file)
fragments = json.load(open(fragments_path, encoding='utf-8'))
quest_graph = book_fragments2graph(fragments, quest_name)

.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(
.
\(
(

In [200]:
quest_graph.number_of_edges()

156

In [220]:
QUESTBOOKS_GRAPH_DIRECTORY = 'D:\Диплом_текстовые_квесты\Data\questbook_graphs'

In [239]:
for file in tqdm(os.listdir(FRAGMENTS_JSON_DIRECTORY)):
    quest_name = os.path.splitext(file)[0]
    fragments_path = os.path.join(FRAGMENTS_JSON_DIRECTORY, file)
    fragments = json.load(open(fragments_path, encoding='utf-8'))
    
    if isinstance(fragments, dict):
        quest_graph = book_fragments2graph(fragments, quest_name)
        quest_graph_json = json_graph.node_link_data(quest_graph)
        with open(os.path.join(QUESTBOOKS_GRAPH_DIRECTORY, file), 'w', encoding='utf-8') as f:
            json.dump(quest_graph_json, f)




  0%|                                                                                          | 0/173 [00:00<?, ?it/s]


  1%|▍                                                                                 | 1/173 [00:00<00:18,  9.53it/s]


  1%|▉                                                                                 | 2/173 [00:00<00:20,  8.45it/s]


  2%|█▍                                                                                | 3/173 [00:00<00:21,  8.05it/s]


  3%|██▎                                                                               | 5/173 [00:00<00:18,  8.96it/s]


  3%|██▊                                                                               | 6/173 [00:01<00:38,  4.32it/s]


  4%|███▎                                                                              | 7/173 [00:02<01:28,  1.88it/s]


  5%|███▊                                                                              | 8/173 [00:02<01:15,  2.20it/s]


  5%|████▎           

In [245]:
graph_statistics = pd.DataFrame(columns=['file', 'num_nodes', 'num_edges', 'num_real_starts'])
for file in os.listdir(QUESTBOOKS_GRAPH_DIRECTORY):
    file_path = os.path.join(QUESTBOOKS_GRAPH_DIRECTORY, file)
    graph_data = json.load(open(file_path, encoding='utf-8'))
    G = json_graph.node_link_graph(graph_data)
    
    num_nodes = G.number_of_nodes()
    num_edges = G.number_of_edges()
    num_real_starts = len([node for node in G.nodes() if G.in_degree(node) == 0 and G.out_degree(node) > 0])
    
    graph_statistics.loc[len(graph_statistics)] = [file, num_nodes, num_edges, num_real_starts]

In [246]:
problem_files = list(graph_statistics[graph_statistics['num_edges'] == 0]['file'])
problem_files

[]

In [244]:
for file in problem_files:
    file_path = os.path.join(QUESTBOOKS_GRAPH_DIRECTORY, file)
    os.remove(file_path)

In [247]:
graph_statistics[graph_statistics['num_real_starts']==1]

,file,num_nodes,num_edges,num_real_starts
3,dafin1.json,130,129,1
4,dogs2.json,115,156,1
10,fairysh.json,59,76,1
20,GD-demo.json,64,154,1
27,"SW4 (A5, v1.0).json",31,30,1
30,zhc.json,141,215,1
33,Алькатрас - П.Горохов.json,113,157,1
38,Большое приключение малыша Лешеньки.json,48,62,1
43,В тени трона.json,84,135,1
54,Город бешеных псов.json,122,194,1


In [248]:
graph_statistics[graph_statistics['num_edges']!=0]

,file,num_nodes,num_edges,num_real_starts
0,BB1rusKalgor.json,73,84,11
1,Black_Panther_v1.2.json,219,313,4
2,Cherniy continent.json,354,612,5
3,dafin1.json,130,129,1
4,dogs2.json,115,156,1
...,...,...,...,...
164,Человекосжималки.json,135,137,3
165,Шерлок Холмс 3 (без фона) v1.1.json,530,881,5
166,Школа для смельчаков.json,101,157,13
167,Эдвард Паккард - Тайна заброшенного замка.json,95,104,1
